# Webscrapen van een API

Het makelijkste om data op te vragen is doormiddel van een [API].  
Veel websites hebben geen openbare API.  
Toch kan het zo zijn dat de website een API gebruikt om data van de client naar de server te sturen.  
Zo kan de website dynamisch _(zonder te refreshen)_ de data op de pagina vernieuwen.   
Hier wordt [XMLHTTPRequests (XHR)] voor gebruikt.

[API]: https://nl.wikipedia.org/wiki/Application_programming_interface
[XMLHTTPRequests (XHR)]: https://nl.wikipedia.org/wiki/XMLHTTP

## Het opzoeken van een API op een website

Open een browser en de Developer Tools.  
In de `Network` tab, zet de opties `Preserve logs`, `Disable cache` aan en filter op `XHR`. 

> ![Developer Tools](./img/devtools_XHR_filter.png "Preserve logs, Disable cache, filter XHR")

Met de browser en de Developer Tools open, navigeer naar de website: https://www.demoblaze.com/  
Dit is een demo website van [Blazemeter] en gebruikt [XHR].  
In de notebook cells hieronder open we de demo website en bekijken we de Network tab voor XHR berichtenverkeer.  

[Blazemeter]: https://www.blazemeter.com/
[XHR]: https://nl.wikipedia.org/wiki/XMLHTTP

In de Developer Tools onder de tab `Network` zie je het dataverkeer en de daarbij horende _HTTP request methods_.  
Om goed te zien wat een website krijgt en verstuurd moet de optie `Preserve log` / `Persist Logs` en de `Disable Cache` aan staan.  
Met deze opties aan behoudt van de `Network` tab alle logs en vraagt de website alle data op die anders werd onthouden in de Cache.  

Met de XHR filter aan zien we alleen dat type request.  
De url https://www.demoblaze.com/config.json is zo een XHR-request wat wordt opgevraagt met de GET method.  

> ![Developer tools XHR get](./img/devtools_XHR_get_request.png "GET request, XHR type response")

Observeer de gegevens die te zien zijn in de `Response data` van dit request.  

Als we er op klikken zien we de data die verstuurd en terug gekregen is.  
Onder de tab `Response` krijgen we de _Raw response data_ te zien.  
Dit kan van alles zijn, HTML, javascript en in dit geval JSON.  

> ![Developer tools XHR response](./img/devtools_XHR_response.png "raw response")

Op de demo website is er een lijst te zien met product catagories.  
Wordt er op de een catagorie geklikt dan wordt deze informatie van de server opgevraagd.  

> ![blaze_api_bycat.png](./img/blaze_api_bycat.png "catagories van demoblaze website")

 Mer de `Network` tab open zien we een POST-request met deze _payload_.  
Een POST request bevat meestal een payload en/of speciale headers, in dit geval is de payload een JSON string.

> ![Developer tools phones payload](./img/blaze_payload_phones.png "raw response")

In de tab `Response` zien we de data die vanuit de server naar de webbrowser is verzonden.  
Dit is wederom JSON data.  
TIP: met de `{}` knop kan de JSON data beter leesbaar worden gemaakt.

> ![blaze_cat_response](./img/blaze_response_phones.png "POST request met JSON payload")


---

## Python gebruiken om de API aan te roepen

Nu de API van de website is gevonden kan deze zelfde data worden opgevraagd via Python.  
Dit kan worden gedaan met de third-party library: [requests].  

De library moet geinstalleerd worden met `pip`  

[requests]: https://docs.python-requests.org/en/master/index.html

In [1]:
!python -m pip install --upgrade pip requests

Importeer `requests` en een hulpfuctie uit urllib`urllib.parse`.  
`urllib` is een library uit Python zelf, dus deze hoeft niet geinstalleerd te worden.

In [2]:
from urllib.parse import urljoin
import requests

Met `urljoin` hoeft er geen rekening te houden met _slashes_ aan het einde van een URL.

In [3]:
base_url = 'https://www.demoblaze.com'
path = 'config.json'
config_url = urljoin(base_url, path)

response = requests.get(config_url)
response

<Response [200]>

Het object `response` van `requests.get` heeft meerdere attributen.

In [4]:
[r for r in dir(response) if not r.startswith('_')]

[...]

`response.content` is de data (in `bytes`) die terug is gekregen van het GET request (Raw response data)

In [5]:
response.content

b'{\n    "API_URL": "https://api.demoblaze.com",\n    "HLS_URL": "https://hls.demoblaze.com"\n}'

Er is JSON data ontvangen.  
De data kan geparsed worden door Python `json` module of door de `response.json()` functie naar een dict.

In [6]:
config_dict = response.json()
config_dict

{'API_URL': 'https://api.demoblaze.com',
 'HLS_URL': 'https://hls.demoblaze.com'}

In [7]:
api_url = config_dict.get('API_URL')
api_url

'https://api.demoblaze.com'

Zoals gezien in de Developer Tools kan het pad _bycat_ toegevoegd worden aan de API url.  
Deze API url verwacht wel een json-payload.  

`requests.post` heeft het keyword-argument `json`, hier aan kan een `dict` gegeven worden die als JSON wordt versuurd in de body van het POST-request.

In [8]:
path = 'bycat'
url = urljoin(api_url, path)

payload = {'cat': 'phone'}

response = requests.post(url, json=payload)
notebook_dict =  response.json()
# notebook_dict

Nu de data is ontvangen kan de data, die er echt nodig is, eruit worden gehaald.  
Deze data kunnen we dan opslaan in CSV-file. 

In [9]:
import csv
from pathlib import Path

In [10]:
filepath = Path.cwd().joinpath('phone_costs.csv')

# open de nieuwe CSV-file in schrijf mode: 'w'
with filepath.open('w', newline='') as file: 
    csv_writer = csv.writer(file)
    
    # loop over de items in de dict
    for item in notebook_dict['Items']:
        if item.get('cat', '').lower() != 'phone':
            continue
        
        # haal de data uit de `dict`.
        title = item.get('title', 'No title')
        price = item.get('price', False)
        # schrijf de data naar een nieuwe regel in de CSV-file
        csv_writer.writerow([title, price])

De nieuwe CSV-file is aangemaakt.  
Deze file kan ook weer gelezen worden door Python

In [11]:
with filepath.open('r') as file:
    for line_no, line in enumerate(file, start=1):
        print(f"{line_no}: {line.strip()}")
        if line_no >= 5:
            break

1: Samsung galaxy s6,360.0
2: Nokia lumia 1520,820.0
3: Nexus 6,650.0
4: Samsung galaxy s7,800.0
5: Iphone 6 32gb,790.0
